In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# import matplotlib.pyplot as plt
# from joblib import Parallel, delayed
# from sklearn.ensemble import RandomForestRegressor

In [8]:
strs_ref = pd.read_csv('./res_superconv_gauss_pt_1_stress_ms_256.csv')
# strs_ref


In [9]:
x = pd.read_csv('./coord_gauss_pt_256.csv')
x = x.loc[:, ~x.columns.str.contains('^Unnamed')]
# x


In [10]:
y = strs_ref
y = y.loc[:, ~y.columns.str.contains('^Unnamed')]
y = y.loc[:, ~y.columns.str.contains('^x')]
y = y.loc[:, ~y.columns.str.contains('^y')]
y.head()


,sigma_x_ref,sigma_y_ref,sigma_xy_ref
0,0.023972,0.014232,0.019413
1,0.028136,0.021371,0.024545
2,0.030626,0.023639,0.027049
3,0.032598,0.025966,0.029111
4,0.034184,0.027419,0.030667


In [11]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []

    nodes_increment = (last_layer_nodes - first_layer_nodes) / (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    return layers


In [12]:
import time
start = time.time()


In [13]:
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    #newmodel
    model1 = Sequential()
    n_nodes = FindLayerNodesLinear(
        n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i == 1:
            model1.add(Dense(first_layer_nodes, activation=activation_func))
        else:
            model1.add(Dense(n_nodes[i-1], activation=activation_func))

    model1.add(Dense(units=3, activation='linear'))
    model1.compile(
        loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
    return model1


seed = 7
np.random.seed(seed)


In [14]:
X = x
Y = y


In [15]:

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [256, 512, 1024, 2048]
epochs = [50, 100, 150, 200, 300]
n_layers = [5, 8, 10]
first_layer_nodes = [50, 60, 80]
last_layer_nodes = [5, 10]
activation_funcs = ['relu', 'sigmoid', 'tanh']
param_grid = dict(epochs=epochs, batch_size=batch_size, n_layers=n_layers,
                  first_layer_nodes=first_layer_nodes, last_layer_nodes=last_layer_nodes, activation_func=activation_funcs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    n_jobs=-1, cv=5, verbose=2)
#print(1, neurons1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


/tmp/ipykernel_73005/1982753214.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)
2023-04-01 16:42:16.496808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 16:42:16.496808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 16:42:16.496808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 16:42:16.496808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 16:42:16.496809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-01 16:42:16.496808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT 

KeyboardInterrupt: 

In [ ]:
end = time.time()
print("The time of execution of above program is :", end-start)
